In [1]:
import pandas as pd
from ekonlpy.tag import Mecab

In [2]:
def listing():
    df = pd.read_csv('./news_temp.csv')
    df = df.dropna() # nan값 제거

    data = df.groupby('date').agg({'title': '/'.join, 'contents': '/'.join}).reset_index() # 날짜별로 뉴스 제목 및 내용 묶기
    melted_data = pd.melt(data, id_vars=['date'], value_vars=['title', 'contents'], var_name='column', value_name='title_contents')
    data = melted_data.groupby('date')['title_contents'].agg('/'.join).reset_index()

    return data

In [3]:
total_news = listing()

# stopPos를 적용한 값을 넣어주기 위해 result값 생성
total_news=total_news.assign(total_pre="",result="")
total_news

,date,title_contents,total_pre,result
0,2017.01.01.,명량해전 이순신처럼… 경제수장들 신년사 고사성어 눈길/치솟는 원 달러 환율에 잠 ...,,
1,2017.01.02.,"광주은행, 모바일 대출 신상품 출시…'최대 1천500만원'/이주열 올해 경제 녹록...",,
2,2017.01.03.,"지자체별로 다른 음식점 인증제도, 올해부터 통일된다/한중일+아세안, 독자 통화스와프...",,
3,2017.01.04.,서울 강남대로 불법 노점 사라지고 휴식 공간 들어섰다/황권한대행 국민체감도 높은 ...,,
4,2017.01.05.,올해 111만가구 주거지원…'주거복지 청사진' 만든다/금감원장 상당수 중기 환율변...,,
5,2017.01.06.,"미국 월간 신규고용 15만6천건으로↓… 고용호조 지속 종합/국제유가, 사우디 감산...",,
6,2017.01.07.,브라질 산업생산 증가세 기대 이하…경제회복 제자리 걸음/뉴욕증시 S P 나스닥 사상...,,
7,2017.01.08.,"인니, 투자은행 시장보고서 검열… 미돈줄죄기에 긴장한 탓/전문가들 유일호 경제팀,...",,
8,2017.01.09.,불황인데 물가만 껑충…한국 '스태그플레이션' 빠지나/'초저금리 약발없나'…증시 맴도...,,
9,2017.01.10.,올해 오피스 시장도 침체…공실 늘고 거래 위축/윤경은 KB증권사장 국민자산증식 파...,,


In [4]:
mecab = Mecab()

for i in range(len(total_news)):
        total_news['total_pre'][i] = mecab.pos(total_news['title_contents'][i]) # total_pre에 pos tagging 값 넣기

In [5]:
total_news

,date,title_contents,total_pre,result
0,2017.01.01.,명량해전 이순신처럼… 경제수장들 신년사 고사성어 눈길/치솟는 원 달러 환율에 잠 ...,"[(명량, NNP), (해전, NNG), (이, JKS), (순신, NNG), (처...",
1,2017.01.02.,"광주은행, 모바일 대출 신상품 출시…'최대 1천500만원'/이주열 올해 경제 녹록...","[(광주은행, NNP), (,, SC), (모바일, NNP), (대출, NNG), ...",
2,2017.01.03.,"지자체별로 다른 음식점 인증제도, 올해부터 통일된다/한중일+아세안, 독자 통화스와프...","[(지자체, NNG), (별, XSN), (로, JKB), (다른, MM), (음식...",
3,2017.01.04.,서울 강남대로 불법 노점 사라지고 휴식 공간 들어섰다/황권한대행 국민체감도 높은 ...,"[(서울, NNP), (강남대로, NNP), (불법, NNG), (노점, NNG),...",
4,2017.01.05.,올해 111만가구 주거지원…'주거복지 청사진' 만든다/금감원장 상당수 중기 환율변...,"[(올해, NNG), (111, SN), (만, NR), (가구, NNG), (주거...",
5,2017.01.06.,"미국 월간 신규고용 15만6천건으로↓… 고용호조 지속 종합/국제유가, 사우디 감산...","[(미국, NNG), (월간, NNG), (신규, NNG), (고용, NNG), (...",
6,2017.01.07.,브라질 산업생산 증가세 기대 이하…경제회복 제자리 걸음/뉴욕증시 S P 나스닥 사상...,"[(브라질, NNG), (산업, NNG), (생산, NNG), (증가세, NNG),...",
7,2017.01.08.,"인니, 투자은행 시장보고서 검열… 미돈줄죄기에 긴장한 탓/전문가들 유일호 경제팀,...","[(인니, NNG), (,, SC), (투자은행, NNG), (시장, NNG), (...",
8,2017.01.09.,불황인데 물가만 껑충…한국 '스태그플레이션' 빠지나/'초저금리 약발없나'…증시 맴도...,"[(불황, NNG), (인데, VCP), (물가, NNG), (만, JX), (껑충...",
9,2017.01.10.,올해 오피스 시장도 침체…공실 늘고 거래 위축/윤경은 KB증권사장 국민자산증식 파...,"[(올해, NNG), (오피스, NNG), (시장, NNG), (도, JX), (침...",


In [6]:
# 불필요하다고 판단 된 품사 제거
# 품사와 품사 태그 셋 : https://happygrammer.github.io/nlp/postag-set/

stopPos= ['NR',
 'VX', 'VCP', 'VCN', 'MM', 'IC',
 'JKC', 'JKO', 'JKB', 'JKV', 'JKQ', 
'JX', 'JC', 'EP', 'EF',
 'EC', 'ETN', 'ETM', 'XPN', 'XSN',
 'XSV', 'XSA', 'XR', 'SF', 'SE',
 'SSO', 'SSC', 'SC', 'SY']

In [15]:
total_news['result'] = [[] for _ in range(len(total_news))]

for n, i in enumerate(total_news['total_pre']):
    for j in i:
        if j[1] not in stopPos:
            total_news['result'][n].append([j])

In [16]:
total_news

,date,title_contents,total_pre,result
0,2017.01.01.,명량해전 이순신처럼… 경제수장들 신년사 고사성어 눈길/치솟는 원 달러 환율에 잠 ...,"[(명량, NNP), (해전, NNG), (이, JKS), (순신, NNG), (처...","[[(명량, NNP)], [(해전, NNG)], [(이, JKS)], [(순신, N..."
1,2017.01.02.,"광주은행, 모바일 대출 신상품 출시…'최대 1천500만원'/이주열 올해 경제 녹록...","[(광주은행, NNP), (,, SC), (모바일, NNP), (대출, NNG), ...","[[(광주은행, NNP)], [(모바일, NNP)], [(대출, NNG)], [(신..."
2,2017.01.03.,"지자체별로 다른 음식점 인증제도, 올해부터 통일된다/한중일+아세안, 독자 통화스와프...","[(지자체, NNG), (별, XSN), (로, JKB), (다른, MM), (음식...","[[(지자체, NNG)], [(음식점, NNG)], [(인증제도, NNG)], [(..."
3,2017.01.04.,서울 강남대로 불법 노점 사라지고 휴식 공간 들어섰다/황권한대행 국민체감도 높은 ...,"[(서울, NNP), (강남대로, NNP), (불법, NNG), (노점, NNG),...","[[(서울, NNP)], [(강남대로, NNP)], [(불법, NNG)], [(노점..."
4,2017.01.05.,올해 111만가구 주거지원…'주거복지 청사진' 만든다/금감원장 상당수 중기 환율변...,"[(올해, NNG), (111, SN), (만, NR), (가구, NNG), (주거...","[[(올해, NNG)], [(111, SN)], [(가구, NNG)], [(주거, ..."
5,2017.01.06.,"미국 월간 신규고용 15만6천건으로↓… 고용호조 지속 종합/국제유가, 사우디 감산...","[(미국, NNG), (월간, NNG), (신규, NNG), (고용, NNG), (...","[[(미국, NNG)], [(월간, NNG)], [(신규, NNG)], [(고용, ..."
6,2017.01.07.,브라질 산업생산 증가세 기대 이하…경제회복 제자리 걸음/뉴욕증시 S P 나스닥 사상...,"[(브라질, NNG), (산업, NNG), (생산, NNG), (증가세, NNG),...","[[(브라질, NNG)], [(산업, NNG)], [(생산, NNG)], [(증가세..."
7,2017.01.08.,"인니, 투자은행 시장보고서 검열… 미돈줄죄기에 긴장한 탓/전문가들 유일호 경제팀,...","[(인니, NNG), (,, SC), (투자은행, NNG), (시장, NNG), (...","[[(인니, NNG)], [(투자은행, NNG)], [(시장, NNG)], [(보고..."
8,2017.01.09.,불황인데 물가만 껑충…한국 '스태그플레이션' 빠지나/'초저금리 약발없나'…증시 맴도...,"[(불황, NNG), (인데, VCP), (물가, NNG), (만, JX), (껑충...","[[(불황, NNG)], [(물가, NNG)], [(껑충, MAG)], [(한국, ..."
9,2017.01.10.,올해 오피스 시장도 침체…공실 늘고 거래 위축/윤경은 KB증권사장 국민자산증식 파...,"[(올해, NNG), (오피스, NNG), (시장, NNG), (도, JX), (침...","[[(올해, NNG)], [(오피스, NNG)], [(시장, NNG)], [(침체,..."


In [17]:
from itertools import chain

total_news['result'] = list(chain.from_iterable(total_news['result']))

ValueError: Length of values (324763) does not match length of index (31)